In [1]:
from pathlib import Path

import util
from util import workflow

browser = False
file = util.notebook_file() if util.is_notebook() else __file__
tag = util.file_tag(file)
root_path = Path("..")

In [2]:
# Build
from automol.graph import enum

import automech
from automech.species import Species

par_mech = workflow.read_parent_mechanism(root_path=root_path)
mech = automech.from_smiles(spc_smis=["C1=CCCC1"], src_mech=par_mech)
#  - add OH addition to *ene*
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.pi2_addition,
    rcts_=[None, "[OH]"],
    spc_col_=Species.smiles,
    src_mech=par_mech,
)
#  - add 1,2 H-migrations (1,3 should be negligible)
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.h_migration_12,
    spc_col_=Species.smiles,
    src_mech=par_mech,
    repeat=2,  # This could maybe be taken out?
)
#  - add ring beta scissions
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.ring_beta_scission,
    src_mech=par_mech,
)
#  - add beta scissions
mech = automech.enumerate_reactions(
    mech,
    enum.ReactionSmarts.beta_scission,
    src_mech=par_mech,
)

  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
# Write
workflow.write(mech=mech, tag=tag, root_path=root_path, browser=browser)


Finalizing build for...
reactions=shape: (17, 5)
┌────────────────┬────────────────┬───────────┬────────────┬─────────────────────────────────┐
│ reactants      ┆ products       ┆ formula   ┆ reversible ┆ rate_constant                   │
│ ---            ┆ ---            ┆ ---       ┆ ---        ┆ ---                             │
│ list[str]      ┆ list[str]      ┆ struct[3] ┆ bool       ┆ struct[17]                      │
╞════════════════╪════════════════╪═══════════╪════════════╪═════════════════════════════════╡
│ ["C5H9O(878)"] ┆ ["C5H9O(852)"] ┆ {5,9,1}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["C5H9O(853)"] ┆ ["C5H9O(852)"] ┆ {5,9,1}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1528)"]    ┆ ["S(1247)"]    ┆ {5,9,1}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1431)"]    ┆ ["S(1248)"]    ┆ {5,9,1}   ┆ true       ┆ {1,{null,null,null,null,null,n… │
│ ["S(1259)"]    ┆ ["S(1248)"]    ┆ {5,9,1}   ┆ true       ┆ {1,{null,null,null,null,null,n… │


  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]


Writing mechanism...
../data/A_rh-oh_p2v1_orig.json
../data/A_rh-oh_p2v1.json
../data/mechanalyzer/A_rh-oh_p2v1.dat
../data/mechanalyzer/A_rh-oh_p2v1.csv


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]


Stereoexpansion errors:


In [4]:
# # Read
# workflow.read(tag=tag, root_path=root_path)

In [5]:
# # Simulate
# workflow.simulate(full_tag=f"full_{tag}_calc", root_path=root_path)
# workflow.simulate(full_tag=f"full_{tag}_control", root_path=root_path)

In [6]:
# import automech
# from mechdriver.subtasks import display, fs

# chan = "1: 1"

# # TRANSITION STATE
# #   - Display the TS mode
# calc_path = util.calc_path(root_path, tag)
# display("find_ts", chan, path=calc_path)

# #   - Show paths
# for path in fs.task_paths("find_ts", chan, path=calc_path):
#     print(path)

# # REACTION RATE
# #   - Read in calculated mechanism
# cal_mech = automech.io.read(data_path / f"{tag}_calc.json")

# #   - Read in other mechanisms for comparison
# par_mech = automech.io.read(data_path / "full_raw.json")
# tags0 = util.previous_tags(tag)
# trues = [True] * len(tags0)
# names0 = list(map(util.calculated_mechanism_name, tags0))
# mechs0 = [automech.io.read(data_path / f"{name}.json") for name in names0]

# #   - Display the reaction and calculated rate
# automech.display_reactions(
#     cal_mech,
#     chans=[chan],
#     comp_mechs=[par_mech, *mechs0],
#     comp_labels=["Hill", *tags0],
#     comp_stereo=[False, *trues],
# )